# Transformations and representations

##### Imports

In [2]:
import numpy as np
from scipy.optimize import fsolve
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'

import matplotlib.pyplot as plt
import math
from pylab import *
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import norm
import copy
import open3d as o3d
import time

import ai2thor.controller
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
import json
from datetime import datetime

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 1. Euler angles

Function that returns a rotation matrix given the angles (𝛼, 𝛽, 𝛾) in radians (X-Y-Z)

In [5]:
pi = np.pi

def rotation_matrix(alpha,beta,gamma):

    R = np.array([[0.72907076 , -0.56745855 ,0.38268343],
                  [0.64785386 , 0.39180184 , -0.65328148],
                  [0.22077409 , 0.72421137 , 0.65328148]])
    # Rotation about x axis
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(alpha), -np.sin(alpha)],
                   [0, np.sin(alpha), np.cos(alpha)]])
    
    # Rotation about y axis    
    Ry = np.array([[np.cos(beta), 0, np.sin(beta)],
                   [0, 1, 0],
                   [-np.sin(beta), 0, np.cos(beta)]])
    
    # Rotation about z axis
    Rz = np.array([[np.cos(gamma), -np.sin(gamma), 0],
                   [np.sin(gamma), np.cos(gamma), 0],
                   [0, 0, 1]])
    
    # Final Rotation matrix

    R_zyx = np.dot(Rz,np.dot(Rx,Ry))

    return R_zyx



In [6]:
a1 = pi/6   # Alpha
a2 = (7*pi)/18 # Beta
a3 = (2*pi)/9 # Gamma

print("Rotation Matrix (𝛼, 𝛽, 𝛾):")
print( rotation_matrix(a1,a2,a3))

Rotation Matrix (𝛼, 𝛽, 𝛾):
[[-0.04000876 -0.5566704   0.82976947]
 [ 0.57976947  0.66341395  0.47302146]
 [-0.81379768  0.5         0.29619813]]


Using fsolve 

In [7]:

def euler_equations(angles):
    # cos =  np.cos
    # sin = np.sin
    # pi = np.pi

    alpha,beta,gamma = angles
    R = np.array([[0.72907076 , -0.56745855 ,0.38268343],
                  [0.64785386 , 0.39180184 , -0.65328148],
                  [0.22077409 , 0.72421137 , 0.65328148]])
    # Rotation about x axis
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(alpha), -np.sin(alpha)],
                   [0, np.sin(alpha), np.cos(alpha)]])
    
    # Rotation about y axis    
    Ry = np.array([[np.cos(beta), 0, np.sin(beta)],
                   [0, 1, 0],
                   [-np.sin(beta), 0, np.cos(beta)]])
    
    # Rotation about z axis
    Rz = np.array([[np.cos(gamma), -np.sin(gamma), 0],
                   [np.sin(gamma), np.cos(gamma), 0],
                   [0, 0, 1]])
    
    # Final Rotation matrix
    R_xyz = np.dot(np.dot(Rx,Ry),Rz)

    result = R_xyz - R
    return [result[0][1], result[0][2], result[1][2]]
    
initial_gu = [0.2,0.2,0.3]



# Initial guess for the Euler angles
initial_guess = [0, 0, 0]

# Solve the system of equations to find Euler angles
euler_angles = fsolve(euler_equations, initial_gu)
print("Euler Angles (𝛼, 𝛽, 𝛾):", euler_angles)

Euler Angles (𝛼, 𝛽, 𝛾): [0.78539816 0.39269908 0.66138792]


## 2. Gimbal Lock

In [3]:
# Load the point cloud
file_path = 'data/toothless.ply'
point_cloud = o3d.io.read_point_cloud(file_path)
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=110, origin=[0, 0, 0])



# Create a visualization window
vis = o3d.visualization.Visualizer()
vis.create_window()

# Add the point cloud to the visualization
vis.add_geometry(point_cloud)

vis.add_geometry(mesh_frame)
# Create coordinate frame geometries for visualization
coord_frame_x = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
coord_frame_y = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
coord_frame_z = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])

# Add coordinate frames to the visualization
vis.add_geometry(coord_frame_x)
vis.add_geometry(coord_frame_y)
vis.add_geometry(coord_frame_z)

# Define animation parameters
num_steps_x = 10  # Number of animation steps for X-axis rotation (90 degrees)
num_steps_y = 10  # Number of animation steps for Y-axis rotation (90 degrees)
num_steps_z = 10  # Number of animation steps for Z-axis rotation (90 degrees)


# Create animation loop for Y-axis rotation (90 degrees)
for _ in range(num_steps_y):
    transformation = np.identity(4)
    rotation_matrix = np.array([[math.cos(math.radians(90 / num_steps_y)), 0, math.sin(math.radians(90 / num_steps_y))],
                                [0, 1, 0],
                                [-math.sin(math.radians(90 / num_steps_y)), 0, math.cos(math.radians(90 / num_steps_y))]])

    transformation[:3, :3] = rotation_matrix

    # Update the transformation of the point cloud
    point_cloud.transform(transformation)
    mesh_frame.rotate(rotation_matrix)

    # Update the visualization
    vis.update_geometry(point_cloud)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)


for _ in range(num_steps_x):
    transformation = np.identity(4)
    rotation_matrix = np.array([[1, 0, 0],
                                [0, math.cos(math.radians(90/num_steps_x)), -math.sin(math.radians(90/num_steps_x))],
                                [0, math.sin(math.radians(90/num_steps_x)), math.cos(math.radians(90/num_steps_x))]])

    transformation[:3, :3] = rotation_matrix

    # Update the transformation of the point cloud
    point_cloud.transform(transformation)
    mesh_frame.rotate(rotation_matrix)
    # Update the visualization
    vis.update_geometry(point_cloud)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)
    # Create animation loop for Y-axis rotation (90 degrees)
for _ in range(num_steps_y):
    transformation = np.identity(4)
    rotation_matrix = np.array([[math.cos(math.radians(90 / num_steps_y)), 0, math.sin(math.radians(90 / num_steps_y))],
                                [0, 1, 0],
                                [-math.sin(math.radians(90 / num_steps_y)), 0, math.cos(math.radians(90 / num_steps_y))]])

    transformation[:3, :3] = rotation_matrix

    # Update the transformation of the point cloud
    point_cloud.transform(transformation)
    mesh_frame.rotate(rotation_matrix)
    # Update the visualization
    vis.update_geometry(point_cloud)
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)


libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


[Open3D WARNING] GLFW Error: GLX: Failed to create context: GLXBadFBConfig
[Open3D WARNING] Failed to create window


## 3. Quaternions

Rotation Matrix to Quaternion Conversion

In [11]:

def rotation_matrix_to_quaternion(R):
    # Ensure the input matrix is a valid rotation matrix (orthogonal and determinant 1)
    if not np.allclose(np.linalg.det(R), 1.0) or not np.allclose(np.dot(R.T, R), np.identity(3)):
        raise ValueError("Input matrix is not a valid rotation matrix.")

    trace = np.trace(R)
    
    if trace > 0:
        S = 2.0 * np.sqrt(trace + 1.0)
        qw = 0.25 * S
        qx = (R[2, 1] - R[1, 2]) / S
        qy = (R[0, 2] - R[2, 0]) / S
        qz = (R[1, 0] - R[0, 1]) / S
    elif R[0, 0] > R[1, 1] and R[0, 0] > R[2, 2]:
        S = 2.0 * np.sqrt(1.0 + R[0, 0] - R[1, 1] - R[2, 2])
        qw = (R[2, 1] - R[1, 2]) / S
        qx = 0.25 * S
        qy = (R[0, 1] + R[1, 0]) / S
        qz = (R[0, 2] + R[2, 0]) / S
    elif R[1, 1] > R[2, 2]:
        S = 2.0 * np.sqrt(1.0 + R[1, 1] - R[0, 0] - R[2, 2])
        qw = (R[0, 2] - R[2, 0]) / S
        qx = (R[0, 1] + R[1, 0]) / S
        qy = 0.25 * S
        qz = (R[1, 2] + R[2, 1]) / S
    else:
        S = 2.0 * np.sqrt(1.0 + R[2, 2] - R[0, 0] - R[1, 1])
        qw = (R[1, 0] - R[0, 1]) / S
        qx = (R[0, 2] + R[2, 0]) / S
        qy = (R[1, 2] + R[2, 1]) / S
        qz = 0.25 * S
    
    return np.array([qw, qx, qy, qz])





In [12]:
# rotation_matrix = np.array([[1,0,0],[0,1,0],[0,0,1]])
rotation_matrix = np.array([[0,-1,0],[1,0,0],[0,0,1]])
# rotation_matrix = np.array([[0,0,1],[0,1,0],[-1,0,0]])

q = rotation_matrix_to_quaternion(rotation_matrix)
print("Quaternion: ",q)


Quaternion:  [0.70710678 0.         0.         0.70710678]


Quaternion to Rotation Matrix

In [10]:


def quaternion_to_rotation_matrix(q):
    qw, qx, qy, qz = q
    R = np.array([
        [1 - 2*qy**2 - 2*qz**2, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw],
        [2*qx*qy + 2*qz*qw, 1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
        [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx**2 - 2*qy**2]])
    return np.around(R,4)


In [9]:
# Example usage:
quaternion = np.array([0.7071, 0.0, 0.0, 0.7071])  # Replace with your quaternion
rotation_matrix = quaternion_to_rotation_matrix(quaternion)
print("Rotation Matrix:\n", rotation_matrix)

Rotation Matrix:
 [[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]


Matrix multiplication of two 3×3 rotation matrices and the same transformation in the quaternion space

In [13]:

# point P in space
P = [1,1,1]
# Define two 3x3 rotation matrices
rotation_matrix1 = np.array([[1,0, 0],
                             [0,0,-1],
                             [0, 1, 0]])

rotation_matrix2 = np.array([[0,0,1],
                             [0,1,0],
                             [-1, 0, 0]])

# Multiply the two rotation matrices R = R1.R2
combined_rotation_matrix = np.dot(rotation_matrix1, rotation_matrix2)
print("Rotation matrix after multiplication is : ")
print(combined_rotation_matrix)

def rotation_matrix_to_quaternion(R):
    # Ensure the input matrix is a valid rotation matrix (orthogonal and determinant 1)
    if not np.allclose(np.linalg.det(R), 1.0) or not np.allclose(np.dot(R.T, R), np.identity(3)):
        raise ValueError("Input matrix is not a valid rotation matrix.")

    trace = np.trace(R)
    
    if trace > 0:
        S = 2.0 * np.sqrt(trace + 1.0)
        qw = 0.25 * S
        qx = (R[2, 1] - R[1, 2]) / S
        qy = (R[0, 2] - R[2, 0]) / S
        qz = (R[1, 0] - R[0, 1]) / S
    elif R[0, 0] > R[1, 1] and R[0, 0] > R[2, 2]:
        S = 2.0 * np.sqrt(1.0 + R[0, 0] - R[1, 1] - R[2, 2])
        qw = (R[2, 1] - R[1, 2]) / S
        qx = 0.25 * S
        qy = (R[0, 1] + R[1, 0]) / S
        qz = (R[0, 2] + R[2, 0]) / S
    elif R[1, 1] > R[2, 2]:
        S = 2.0 * np.sqrt(1.0 + R[1, 1] - R[0, 0] - R[2, 2])
        qw = (R[0, 2] - R[2, 0]) / S
        qx = (R[0, 1] + R[1, 0]) / S
        qy = 0.25 * S
        qz = (R[1, 2] + R[2, 1]) / S
    else:
        S = 2.0 * np.sqrt(1.0 + R[2, 2] - R[0, 0] - R[1, 1])
        qw = (R[1, 0] - R[0, 1]) / S
        qx = (R[0, 2] + R[2, 0]) / S
        qy = (R[1, 2] + R[2, 1]) / S
        qz = 0.25 * S
    
    return np.array([qw, qx, qy, qz])


# combined_quaternion_direct = np.quaternion(quaternion1) * np.quaternion(quaternion2)
p = rotation_matrix_to_quaternion(rotation_matrix1)
q = rotation_matrix_to_quaternion(rotation_matrix2)
# Multiply the two quaternions directly
def quaternion_multiply(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    qw = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    qx = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    qy = w1 * y2 - x1 * z2 + y1 * w2 + z1 * x2
    qz = w1 * z2 + x1 * y2 - y1 * x2 + z1 * w2
    return np.array([qw, qx, qy, qz])


combined_quaternion = quaternion_multiply(p,q)
combined_quaternion_direct = rotation_matrix_to_quaternion(combined_rotation_matrix)
print("Combined Quaternion (Matrix Conversion):", combined_quaternion)
print("Combined Quaternion (Direct Multiplication):", np.array(combined_quaternion_direct))

# Check if the two transformations are the same (quaternion multiplication is non-commutative)
if np.allclose(combined_quaternion, combined_quaternion_direct):
    print("Transformations are the same.")
else:
    print("Transformations are not the same.")


Rotation matrix after multiplication is : 
[[0 0 1]
 [1 0 0]
 [0 1 0]]
Combined Quaternion (Matrix Conversion): [0.5 0.5 0.5 0.5]
Combined Quaternion (Direct Multiplication): [0.5 0.5 0.5 0.5]
Transformations are the same.
